In [1]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

from sql_assistant.config import path_db, chat, coder
from sql_assistant.utils import load_llm_chat
load_dotenv()

True

In [2]:
chat_llm = load_llm_chat(chat)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

C:\Users\jotap\anaconda3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jotap\.cache\huggingface\hub\models--microsoft--Phi-3.5-mini-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [4]:
coder_llm = load_llm_chat(coder)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

C:\Users\jotap\anaconda3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jotap\.cache\huggingface\hub\models--Qwen--Qwen2.5-Coder-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
chat_llm.invoke("Hello")

AIMessage(content="Hello! I'm Phi, an AI developed by Microsoft. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=23, prompt_tokens=4, total_tokens=27), 'model': '', 'finish_reason': 'stop'}, id='run-3a356c01-bfdf-49eb-a4df-8418f15709e6-0')

In [5]:
coder_llm.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=30, total_tokens=40), 'model': '', 'finish_reason': 'stop'}, id='run-8ad42af9-25e5-467b-b811-d6148e8f2ac6-0')

In [16]:
db = SQLDatabase.from_uri(f"sqlite:///{path_db}")
conn = sqlite3.connect(path_db)
print(db.get_usable_table_names())

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [20]:
query_artists = "SELECT * FROM artists;"
query_customers = "SELECT * FROM customers;"
query_employees = "SELECT * FROM employees;"
query_invoices = "SELECT * FROM invoices;"
query_invoices_items = "SELECT * FROM invoice_items;"

In [17]:
query_test = """
SELECT 
    employees.FirstName,
    employees.LastName,
    COUNT(*) as UnitsSold
FROM employees
JOIN invoices
    ON employees.EmployeeId = invoices.EmployeeId
GROUP BY employees.FirstName, employees.LastName;
"""

In [22]:
df = pd.read_sql_query(query_invoices, conn)
df

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [18]:
df = pd.read_sql_query(query_test, conn)
df

DatabaseError: Execution failed on sql '
SELECT 
    employees.FirstName,
    employees.LastName,
    COUNT(*) as UnitsSold
FROM employees
JOIN invoices
    ON employees.EmployeeId = invoices.EmployeeId
GROUP BY employees.FirstName, employees.LastName;
': no such column: invoices.EmployeeId

In [10]:
items_df = pd.read_sql_query(query_invoices_items, conn)
items_df

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
...,...,...,...,...,...
2235,2236,411,3136,0.99,1
2236,2237,411,3145,0.99,1
2237,2238,411,3154,0.99,1
2238,2239,411,3163,0.99,1


In [5]:
db.run("PRAGMA table_info(artists);")

"[(0, 'ArtistId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(120)', 0, None, 0)]"